In [1]:
from is_economic_model.train_model import EconomicClassifier  #TODO
from is_economic_model.tdm_parser import TdmXmlParser #TODO
from sentiment_model.sentiment_score import TextAnalysis #TODO
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import joblib
from tqdm.notebook import tqdm  # Import tqdm for Jupyter Notebook
from joblib import Parallel, delayed



In [2]:

            
def read_file_names_in_chunks(input_file, chunk_size):
    """
    Open a text file and read each row as a file name in chunks.
    
    Args:
        input_file (str): The path to the input text file.
        chunk_size (int): The number of lines (file names) to read per chunk.
        
    Yields:
        list: A list of file names read from the text file, chunk by chunk.
    """
    with open(f'{input_file}.txt', 'r') as f:
        chunk = []
        for i, line in enumerate(f, 1):
            file_name = line.strip()  # Remove any leading/trailing whitespace/newlines
            if file_name:  # Check if line is not empty
                chunk.append(file_name)
            
            if i % chunk_size == 0:
                yield chunk
                chunk = []  # Reset chunk list for the next batch

        # Yield any remaining lines in the last chunk
        if chunk:
            yield chunk


def xml_to_df(file_name, parser):
    """Read the dataset's XML file and add return into a dict.

    Args:
        dataset_path (str): Path to folder containing the dataset of articles in XML files.
        xml_file_list (list): list of the xml files containing the articles 
    Returns:
        pd.DataFrame: A DataFrame containing the concatenated data from the XML files.
    """
    # Iterate over XML files in the dataset
    file_path = f'{data_path + file_name}'
    soup = parser.get_xml_soup(file_path)
    #print(soup)
    PROPERTY_TAGS = ['GOID', 'SortTitle', 'NumericDate',
                'DocSection', 'mstar', 
                'GenSubjTerm', 'is_economic',
                'text_blob_sentiment']
    
    PROPERTY_NAMES = ['GOID', 'Publisher', 'Date', 
                 'Section', 'Type', 
                 'Tags', 'is_economic',
                 'text_blob_sentiment']

    content_dict = parser.get_xml_to_dict(soup, text=False, property_tags=PROPERTY_TAGS, property_names=PROPERTY_NAMES)  # get content of the file into a dict
    return content_dict


dataset_name_list = ['TheNewYorkTimes_sample20', 'USAToday_sample20', 'LosAngelesTimes_sample20', 'TheWashingtonPost_sample25'] #TODO
project_path = Path('c:/Users/97253/OneDrive/Documents/work/bank of israel/financial division/yossi/tdm sentiment/tdm-sentiment/') #TODO
data_path = project_path / 'data/'



### modify sentiment

In [ ]:
def modify_sentiment(file_path, parser, analyzer):
    """
    Modify an XML file by adding specific tags.

    Args:
        file_path (str): Path to the XML file to be modified.
        parser (TdmXmlParser): Parser instance to handle XML operations.
    """
    try:
        soup = parser.get_xml_soup(file_path) # get soup
        texts = parser.get_art_text(soup) # get article text
        
        # make sure is_economic tag is 1
        is_economic_value = int(soup.find('is_economic').text)
        if is_economic_value != 1: 
            print('Houston, we have a problem!')
            return None

        # modify text_blob_sentiment
        value = analyzer.analyze_article_sentiment(texts, method='textblob') 
        soup = parser.modify_tag(soup, value=value, tag_name='text_blob_sentiment')
        # modify bert_sentiment
        value = analyzer.analyze_article_sentiment(texts, method='bert') 
        soup = parser.modify_tag(soup, value=value, tag_name='bert_sentiment')
        
        # Write the modified XML back to the file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(str(soup))
    except:
        return None
    
    
# add is_economic and text_blob_sentiment tags and write txt file with economic file names 
parser = TdmXmlParser()
classifier = EconomicClassifier()
analyzer = TextAnalysis()

chunk_size = 100#00 #TODO # Adjust chunk size based on your memory and performance needs
# Split the file list into chunks
for i, file_chunk in enumerate(read_file_names_in_chunks(data_path / 'economic_dataset_file_names', chunk_size)):
    chunk_paths = [data_path / 'data_sample' / file_name for file_name in file_chunk]
    #for file_name in file_chunk:
        #path = data_path / 'data_sample' / file_name
        #print(modify_sentiment(path, parser, analyzer))
    with tqdm(total=len(file_chunk)) as pbar:
        # Iterate over XML files in the dataset
        Parallel(n_jobs=-1, backend='threading')(delayed(modify_sentiment)(path, parser, analyzer) for path in chunk_paths)
        
        # Update the progress bar
        pbar.update(len(file_chunk))
